In [2]:
### Generate a tileConfig for BigStitcher

import json
from itertools import product, repeat, chain

# from https://docs.python.org/3/library/itertools.html
def ncycles(iterable, n):
    "Returns the sequence elements n times"
    return chain.from_iterable(repeat(tuple(iterable), n))

params_file = "/Volumes/davidh-ssd/mv-sim/sim3_1/sim3_1.json"
tile_conf_file = "/Volumes/davidh-ssd/mv-sim/sim3_1/tileConf.txt"

with open(params_file, 'r') as fd:
    params = json.load(fd)

with open(tile_conf_file, 'w') as fd:
    fd.writelines('dim=3\n')
    for (i,(x,y)) in enumerate(ncycles(product(range(len(params['x_locs'])), range(len(params['y_locs']))),4)):
        x_loc=params['x_locs'][x] // params['downsampling']
        y_loc=params['y_locs'][y] // params['downsampling']
        fd.writelines('{};;({},{},0)\n'.format(i,x_loc,y_loc))